# GPT playground

Implementation of a Decoder only transformer with self-attention as a basic language model

Following the tutorial Let's build GPT, from sratch: https://youtu.be/kCc8FmEb1nY?si=0wP9Tk7SHDYwhlqh

In [1]:
import os
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
torch.manual_seed(1337)

In [3]:
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 64 # what is the maximum context length for predictions?
max_iters = 5_000
eval_interval = 500
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 100
n_head = 6
n_embd = n_head * 16
n_layer = 6
dropout = 0.2
lr=5.0e-4

In [4]:
imput_path = './data/input.txt'
if not os.path.exists('./data/input.txt'):
    !wget -P ./data https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [5]:
# load the text
with open('./data/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [6]:
print(f"length of dataset in characters: \n{len(text)}\n")
print(f"Sample text: \n{text[:100]}")

length of dataset in characters: 
1115394

Sample text: 
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [7]:
# unique characters in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [8]:
# simple tokenization: a map from characters to integers
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [9]:
# encode the text and create a torch tensor
data = torch.tensor(encode(text), dtype=torch.int64)
print(data.shape, data.dtype)
print(data[:15], decode(data[:15].numpy()))

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0]) First Citizen:



In [10]:
# train and validation split
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

### Self-attention mechanism

In [11]:
B, T, C = 4, 8, 2 # batch, time (block size), channels (vocab size)
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [12]:
# We want a simple "average" self-attention of the previous
# words on each word.
# For each word t in [0,T-1], we want the mean of i=0,1,..,t
# bow: bag of words
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, dim=0) # average over time, size: C

In [13]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [14]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [15]:
# do it more efficiently using a lower triangular matrix multiplication
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (T, T) @ (B, T, C) -> (B, T, T) @ (B, T, C) -> (B, T, C)
xbow2[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [16]:
# the third way, using the softmax function
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [17]:
wei = F.softmax(wei, dim=1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [18]:
xbow3 = wei @ x
torch.allclose(xbow2, xbow3)

True

A good tutorial on transformers is available at

https://www.youtube.com/watch?v=zxQyTK8quyY&list=PLblh5JKOoLUICTaGLRoHQDuF_7q2GfuJF&index=90

query: what public info am I looking for?

key: what public info do I contain?

value: what private info do I have?

In [19]:
# the last version of self-attention
# still assuming info is only available from the previous
# tokens only, and not future ones
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

# a single-head self-attention
_head_size = 16
key = nn.Linear(C, _head_size, bias=False)
query = nn.Linear(C, _head_size, bias=False)
value = nn.Linear(C, _head_size, bias=False)
k = key(x) # (B, T, _head_size)
q = query(x) # (B, T, _head_size)

wei = q @ k.transpose(-2, -1) * _head_size**-0.5 # (B, T, _head_size) @ (B, _head_size, T) -> (B, T, T)
tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)
# out = wei @ x
v = value(x)
out = wei @ v
out.shape

torch.Size([4, 8, 16])

In [20]:
wei.var()

tensor(0.0288, grad_fn=<VarBackward0>)

In [21]:
wei[0]

tensor([[0.1444, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1489, 0.1691, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1112, 0.1166, 0.2001, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1295, 0.1058, 0.2409, 0.1214, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1255, 0.1424, 0.1248, 0.3043, 0.2427, 0.0000, 0.0000, 0.0000],
        [0.1121, 0.1978, 0.0903, 0.2124, 0.3318, 0.3324, 0.0000, 0.0000],
        [0.1226, 0.0618, 0.2802, 0.1506, 0.1965, 0.2635, 0.3510, 0.0000],
        [0.1057, 0.2065, 0.0637, 0.2113, 0.2290, 0.4041, 0.6490, 1.0000]],
       grad_fn=<SelectBackward0>)

In [22]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [23]:
class Head(nn.Module):
    """
    A single head of decoder only self-attention
    """
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)

        # parameters that are saved in the state_dict, but 
        # not trained by the optimizer are registered as buffers
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)

        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)

        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

In [24]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [25]:
class FeedFoward(nn.Module):
    """
    A simple linear layer followed by a non-linearity
    """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd), # feed forward
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd), # projection, going back to residual path
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [26]:
class Block(nn.Module):
    """
    Transformer block: communication followed by computation
    """
    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size) # communication
        self.ffwd = FeedFoward(n_embd) # computation
        
        self.ln1 = nn.LayerNorm(n_embd) # to avoid vanishing/exploding gradients
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        # pre-normalize the layers before getting the attention and feed-forward
        x = x + self.sa(self.ln1(x)) # add residual path
        x = x + self.ffwd(self.ln2(x)) # add residual path
        return x


In [27]:
class GPTLanguageModel(nn.Module):
    """
    B: batch_size
    T: time: block_size
    C: channel or n_embd
    """
    def __init__(self):
        super().__init__()
        # each token reads the logits for the next token
        # from a torch lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better initialization
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B, T) shape integer tensors
        tok_emb = self.token_embedding_table(idx) # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B, T, C)
        
        x = self.blocks(x) # apply self attention, (B, T, C)
        x = self.ln_f(x) # (B,T,C)
        
        logits = self.lm_head(x) # (B, T, vocab_size)
        
        if targets is None:
            loss = None
        else:
            # convert to (B*C, T) for log-likelihood loss calculation
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_tokens):
        """
        Generates (B, T+1), (B, T+2), ..., (B, T+max_tokens)
            with the next characters generated given the context
        
        To be able to generate text using the model
        idx: the current context array of characters in a batch of size (B, T)
        """
        for _ in range(max_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]

            # get the predictions
            logits, loss = self(idx_cond) # (B, T, C)

            # last time step is the predictor of the next character
            # since this is a bigram model
            logits = logits[:, -1, :] # (B, C)

            # calculate the probabilities along the channel
            probs = F.softmax(logits, dim=-1) # (B, C)

            # sample from the probability distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            
            # append sampled index to the running sequence
            # along the time dimension
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

        return idx

In [28]:
model = GPTLanguageModel()
m = model.to(device)

print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

optimizer = torch.optim.AdamW(m.parameters(), lr=lr)

0.688193 M parameters


In [29]:
for iter in range(max_iters):
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.1740, val loss 4.1736
step 500: train loss 2.1854, val loss 2.2113
step 1000: train loss 1.8971, val loss 1.9832
step 1500: train loss 1.7366, val loss 1.8761
step 2000: train loss 1.6542, val loss 1.8246
step 2500: train loss 1.6051, val loss 1.7683
step 3000: train loss 1.5649, val loss 1.7394
step 3500: train loss 1.5204, val loss 1.6981
step 4000: train loss 1.4969, val loss 1.6778
step 4500: train loss 1.4873, val loss 1.6818
step 4999: train loss 1.4801, val loss 1.6727


In [30]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_tokens=500)[0].tolist()))



HENRY VINCENIO:
As my smirt Stervants doth to withins for theer.

ISABELLA:
'Sad! o come too, JitLIA:
O but as they:' scans, and unluwed
Which ones ways again unstaintagent, Boietinence.

LEONTUS:
Ay, mays, sweet she 'Preason Montain, showord myself.
Rome, will night till tears, back to desaid
Which the of thus a serving, and is a let valle:
O! O, that not will that most has dote, by daugh
Abortain'st this docks majoon
By my bother, if t, will giving to the chily.
Make the mucketordon.
COMINIUS
